In [59]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn import datasets



In [44]:
#Loading Dataset...
tweets_df = pd.read_csv('clean_tweets_v2.csv')
tweets_df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_tweets
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),said
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),plus added commercials experience tacky
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),today must mean need take another trip
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),really aggressive blast obnoxious entertainm...
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),really big bad thing


In [45]:
tweets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14640 entries, 0 to 14639
Data columns (total 16 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      14640 non-null  int64  
 1   airline_sentiment             14640 non-null  object 
 2   airline_sentiment_confidence  14640 non-null  float64
 3   negativereason                9178 non-null   object 
 4   negativereason_confidence     10522 non-null  float64
 5   airline                       14640 non-null  object 
 6   airline_sentiment_gold        40 non-null     object 
 7   name                          14640 non-null  object 
 8   negativereason_gold           32 non-null     object 
 9   retweet_count                 14640 non-null  int64  
 10  text                          14640 non-null  object 
 11  tweet_coord                   1019 non-null   object 
 12  tweet_created                 14640 non-null  object 
 13  t

In [46]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')

stops = stopwords.words('english')
for i, stop in enumerate(stops):
    stops[i] = re.sub(r'[^a-z]+','', stop)

stops = set(stops)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/zubairlakhia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/zubairlakhia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [47]:
from nltk.tokenize import word_tokenize

def clean_tweet(tweet:str) -> str:
    # remove twitt handles & non-unicode chars
    clean_tweet = re.sub(r'(@[\w]*|[^\x00-\x7F]+)', ' ', tweet)

    # remove punctuation and any non letter characters
    clean_tweet = re.sub(r'[^a-zA-Z\s]+', '', clean_tweet)
    
    # remove links and make lowercase
    clean_tweet = re.sub(r'(www|http)[a-zA-Z]*', '', clean_tweet).lower()

    # if words are present in stop words set then they're removed
    word_tokens = word_tokenize(clean_tweet)
    filtered_words = [word for word in word_tokens if not word in stops]
    
    clean_tweet = " ".join(filtered_words)
    
    return clean_tweet

In [48]:
tweets_df['clean_tweets'] = tweets_df['text'].apply(clean_tweet)


In [49]:
x = df.clean_tweets
y = df.airline_sentiment

print(len(x), len(y))

10445 10445


In [52]:
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size = 0.10, random_state=42)
print(len(x_train), len(y_train))
print(len(x_test), len(y_test))

9400 9400
1045 1045


In [53]:
# instantiate the vectorizer
vect = CountVectorizer()
vect.fit(x_train)

CountVectorizer()

In [55]:
x_train_dtm = vect.transform(x_train)
x_test_dtm = vect.transform(x_test)

In [56]:
vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)
vect_tunned

CountVectorizer(max_df=0.7, max_features=100, min_df=0.1, ngram_range=(1, 2),
                stop_words='english')

In [57]:

model = SVC(kernel='linear', random_state = 10)
model.fit(x_train_dtm, y_train)
#predicting output for test data
pred = model.predict(x_test_dtm)

In [58]:
accuracy_score(y_test,pred)

0.8698564593301435

In [60]:
#K fold Implementation
model = SVC(kernel='linear', random_state = 10)
x = vect.transform(x)
kfold = KFold(n_splits=10, shuffle=True, random_state=42)
accuracy_scores = cross_val_score(model, x, y, cv=kfold, scoring='accuracy')
accuracy_scores

array([0.86985646, 0.84019139, 0.85263158, 0.83827751, 0.84880383,
       0.86685824, 0.84291188, 0.84770115, 0.84099617, 0.86494253])

In [61]:
print(f'Average Accuracy: {accuracy_scores.mean()}')

Average Accuracy: 0.8513170727236062
